# MBTI Parallel Classification Model with LSTM

First, load libraries and useful functions from class:

In [55]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import json, os, re, shutil, sys, time
from importlib import reload
import collections, itertools

# NumPy and TensorFlow
import numpy as np
import pandas as pd
import tensorflow as tf
assert(tf.__version__.startswith("1."))

# Utils and Helper libraries
# import nltk
from w266_common import utils, vocabulary
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

## Specifications for Baseline LSTM for MBTI

In this baseline, the task is to predict the MBTI type (16 types) given a text string. We will model after the A3 assignment, with Architecture and Parameters defined below.

### Pre-Processing:
* Minimial pre-processing, only separating punctuation from text and lower-case all text
* Assigning words to numerical indices based on a fixed Vocab size, defined by word fre-quency in training set

### Architecture:
* Encoder: 1 layer LSTM
* Decoder: Affine on the LSTM output with Softmax on logits
* Classification: 16 MBTI types

### Parameters
* Batch Size: 500 (50 for pilot)
* Text length: 100
* Vocabulary size (V): ~330K - full corpus
* Embedding Size: 300
* Hidden Dimensions: 300

### Training:
* Epochs = 25 (5 for pilot)
* 80% train, 20% test
* Loss: Cross Entropy with Adam initialization
* Optimizers: Adam Optimizer

## Load Corpus & Pre-Process

In [3]:
#load data
df = pd.read_csv('./mbti_1.csv')
df.head(5)

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [4]:
# function to clean and tokenize sentence ["Hello world."] into list of words ["hello world"]
def clean(sentence):
    ignore_words = []
    words = re.sub("[^\w]", " ",  sentence).split() #nltk.word_tokenize(sentence)
#     words = sentence.split() #nltk.word_tokenize(sentence)
    words_cleaned = sentence.lower()
    return words_cleaned

In [5]:
# split posts per users into separate sentences
post = []
utype = []
user = []

for index, row in df.iterrows():
    posts = row['posts'].split('|||')
    posts_clean = []
    for sentence in posts:
        posts_clean.append(clean(sentence))
    post.extend(posts_clean)
#     post.extend(posts)
    utype.extend([row['type'] for i in range(len(posts))])
    user.extend([index for i in range(len(posts))])
    
short_posts = pd.DataFrame({"user": user,"type": utype,"post": post})
print(short_posts.shape)
short_posts.head(5)

(422845, 3)


,post,type,user
0,'http://www.youtube.com/watch?v=qsxhcwe3krw,INFJ,0
1,http://41.media.tumblr.com/tumblr_lfouy03pma1q...,INFJ,0
2,enfp and intj moments https://www.youtube.com...,INFJ,0
3,what has been the most life-changing experienc...,INFJ,0
4,http://www.youtube.com/watch?v=vxzeywwrdw8 h...,INFJ,0


In [6]:
# Split data: 80% train, 20% test
post_train, post_test, label_train, label_test = train_test_split(np.array(short_posts['post']), 
                                                    np.array(short_posts['type']), 
                                                    test_size=0.2, 
                                                    random_state=88)

print("MBIT posts", post_train[:5])
print('')
print("MBTI Labels: ",label_train[:5])

MBIT posts ['https://www.youtube.com/watch?v=bxvkaah2d7m'
 'isfjs and infps can balance each other really well, i think, if they learn to communicate - sjs choosing words (and tone of voice!) carefully, and nps learning to take things less personally. your sj...'
 "i'd seek recognition. not fame."
 "to be honest, maybe you are giving out vibes that you are not self-assured. because if you think about it, a bully is going to go for someone they don't think will fight back, someone who is weak (i..."
 "probably. any thinking, really. personally, i prefer the ax-b-c-dy function stack compared to grant's. it allows for the parts where grant gets it right while still being consistent with jung, i.e.,..."]

MBTI Labels:  ['INTP' 'INFP' 'INTP' 'ENFP' 'INTJ']


In [7]:
# Build a vocabulary (V size is defaulted to full text) for train corpus
vocab_mbti = vocabulary.Vocabulary((utils.canonicalize_word(w) for w in post_train))
vocab_mbti.size

333007

In [8]:
print (vocab_mbti.words_to_ids(['a','what','and','the']))
print (vocab_mbti.ids_to_words([202, 147565, 317206, 159348])) 

[202, 147565, 317206, 159348]
['a', 'what', 'and', 'the']


In [9]:
# tokenize and canonicalize train and test sets
x_train = []
for post in post_train:
    x_train.append(vocab_mbti.words_to_ids(post.split()))

x_test = []
for post in post_test:
    x_test.append(vocab_mbti.words_to_ids(post.split()))

In [10]:
print("Original Text: ",post_train[88])
print("Canonicalized Text: ", x_train[88])
print("Max lengths of texts: ", max([len(x) for x in x_train+x_test]))

Original Text:  i don't agree with brutal..
Canonicalized Text:  [308, 2, 361, 187384, 2]
Max lengths of texts:  66


In [69]:
# 4 dichotomomies: {Attitude: E/I, Information: N/S , Decision: F/T, Structure: J/P }
mbti_map = {'E':0, 'I':1, 'N':2, 'S':3, 'F':4, 'T':5, 'J':6,'P':7 } # indicates the the index column for each type

# split type string --> ['I' 'N' 'T' 'P']
dichotomy_train = np.array([list(mbti_type) for mbti_type in label_train])
dichotomy_test = np.array([list(mbti_type) for mbti_type in label_test])

# convert dichotomy indicators to index value --> [1 2 5 7]
y_train_id = np.array([list(map(lambda x: mbti_map[x],ind)) for ind in dichotomy_train])
y_test_id = np.array([list(map(lambda x: mbti_map[x],ind)) for ind in dichotomy_test])

# create 1-hot vector with MBTI dichotomies, map index value to 1 --> [0,1,1,0,1,0,1,0]
one_hot_train = np.zeros((len(y_train_id),8),dtype=int)
one_hot_test = np.zeros((len(y_test_id),8),dtype=int)
for i in range(4):
    one_hot_train[np.arange(len(dichotomy_train)), y_train_id[:,i]] = 1
    one_hot_test[np.arange(len(dichotomy_test)), y_test_id[:,i]] = 1
y_train = one_hot_train
y_test = one_hot_test

print(y_train_id[:2])
print(y_train[:2])
print(y_test_id[:2])
print(y_test[:2])

[[1 2 5 7]
 [1 2 4 7]]
[[0 1 1 0 0 1 0 1]
 [0 1 1 0 1 0 0 1]]
[[1 2 4 7]
 [1 2 4 6]]
[[0 1 1 0 1 0 0 1]
 [0 1 1 0 1 0 1 0]]


## Bulid the LSTM Model

In [71]:
# Model Parameters
V = vocab_mbti.size
classes = 8 #len(set(labels))
batch_size = 500
text_length = 100 # fixed max length of text, but need to handle truncating
embed_dim = 300 #256
hidden_dims = 300 # Needs to be the same dims for LSTM [256,256]
num_layers = 1

# Training Parameters
dropout_keep_prob = .5
learning_rate = .001
num_epochs = 10

In [72]:
# useful functions for building LSTM
def MakeFancyRNNCell(hidden_dims, keep_prob, layers=1, isTrain=True):
    """Make a fancy RNN cell.

    Use tf.nn.rnn_cell functions to construct an LSTM cell.
    Initialize forget_bias=0.0 for better training.

    Args:
      H: hidden state sizes, provided in array
      keep_prob: dropout keep prob (same for input and output)

    Returns:
      (tf.nn.rnn_cell.RNNCell) multi-layer LSTM cell with dropout
    """
    if isTrain == False:
        keep_prob = 1.0

    cells = []
    for _ in range(layers):
#         cell = tf.nn.rnn_cell.BasicLSTMCell(H, forget_bias=0.0) #deprecated?
        cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_dims, forget_bias=0.0,state_is_tuple=True)
        cell = tf.nn.rnn_cell.DropoutWrapper(cell, 
                                             input_keep_prob=keep_prob, 
                                             output_keep_prob=keep_prob)
        cells.append(cell)
    return tf.nn.rnn_cell.MultiRNNCell(cells)


# def matmul3d(X, W):
#     """Wrapper for tf.matmul to handle a 3D input tensor X.
#     Will perform multiplication along the last dimension.

#     Args:
#       X: [m,n,k]
#       W: [k,l]

#     Returns:
#       XW: [m,n,l]
#     """
#     Xr = tf.reshape(X, [-1, tf.shape(X)[2]])
#     XWr = tf.matmul(Xr, W)
#     newshape = [tf.shape(X)[0], tf.shape(X)[1], tf.shape(W)[1]]
#     return tf.reshape(XWr, newshape)

In [89]:
tf.reset_default_graph()
tf.set_random_seed(88)
    
# Create input placeholder
with tf.name_scope("Inputs"):
    x_text_ = tf.placeholder(tf.int32, [None, None], name="x_text") #batch x text_length
    y_type_ = tf.placeholder(tf.int32, [None,classes], name="y_type") #batch x classes

    
# Model params
with tf.name_scope("Dynamic_Params"):
    # Get dynamic shape info from inputs
    batch_size_ = tf.shape(x_text_)[0]
    ns_ = tf.tile([text_length], [batch_size_, ], name="ns")
    isTrain_ = tf.placeholder(tf.bool, shape=())

# Construct embedding layer
with tf.name_scope("Embedding_Layer"):
#     W_in_ = tf.get_variable(tf.random_uniform(-1.0, 1.0),shape=[V, embed_dim], name="W_in")
    W_in_ = tf.Variable(tf.random_uniform([V, embed_dim], -1.0, 1.0), name="W_in")
    x_ = tf.nn.embedding_lookup(W_in_, x_text_,name='x')
    print("Embedded Input: ", x_.shape)

# Construct RNN/LSTM cell and recurrent layer.
with tf.name_scope("Recurrent_Layer"):
    # 1 layer LSTM
    cell_lstm_ = MakeFancyRNNCell(hidden_dims, dropout_keep_prob, num_layers,isTrain_)
    initial_h_ = cell_lstm_.zero_state(batch_size_, dtype=tf.float32)
    output_, final1_h_= tf.nn.dynamic_rnn(cell=cell_lstm_, inputs=x_, 
                                         sequence_length= ns_, initial_state = initial_h_, dtype=tf.float32)
    # 4 parallel x 1 Layer LSTM
#     cell_lstm1_ = MakeFancyRNNCell(hidden_dims, dropout_keep_prob, num_layers,isTrain_)
#     cell_lstm2_ = MakeFancyRNNCell(hidden_dims, dropout_keep_prob, num_layers,isTrain_)
#     cell_lstm3_ = MakeFancyRNNCell(hidden_dims, dropout_keep_prob, num_layers,isTrain_)
#     cell_lstm14_ = MakeFancyRNNCell(hidden_dims, dropout_keep_prob, num_layers,isTrain_)
#     initial_h_ = cell_lstm1_.zero_state(batch_size_, dtype=tf.float32)
#     output1_, final1_h_= tf.nn.dynamic_rnn(cell=cell_lstm1_, inputs=x_, 
#                                          sequence_length= ns_, initial_state = initial_h_, dtype=tf.float32)
#     output2_, final1_h_= tf.nn.dynamic_rnn(cell=cell_lstm1_, inputs=x_, 
#                                      sequence_length= ns_, initial_state = initial_h_, dtype=tf.float32)
#     output3_, final1_h_= tf.nn.dynamic_rnn(cell=cell_lstm1_, inputs=x_, 
#                                          sequence_length= ns_, initial_state = initial_h_, dtype=tf.float32)
#     output4_, final1_h_= tf.nn.dynamic_rnn(cell=cell_lstm1_, inputs=x_, 
#                                          sequence_length= ns_, initial_state = initial_h_, dtype=tf.float32)
    print("LSTM Cell output shape: ",output_.shape)

with tf.name_scope("Output_Layer"):
    output_ = tf.reshape(output_, [batch_size_,text_length*hidden_dims])
    print("flattened output shape: ",output_.shape)
    W_out_ = tf.Variable(tf.random_uniform([hidden_dims*text_length,classes],-1.0, 1.0), name="W_out")
    print("W_out: ",W_out_.shape)
    b_out_ = tf.Variable(tf.zeros([classes,], dtype=tf.float32), name="b_out")
    logits_ = tf.add(tf.matmul(output_,W_out_), b_out_, name="logits")
    print("Logits: ",logits_.shape)
#     # 4 parallele outputs
#     output1_ = tf.reshape(output1_, [batch_size_,text_length*hidden_dims])
#     output2_ = tf.reshape(output2_, [batch_size_,text_length*hidden_dims])
#     output3_ = tf.reshape(output3_, [batch_size_,text_length*hidden_dims])
#     output4_ = tf.reshape(output4_, [batch_size_,text_length*hidden_dims])
#     output_ = tf.concat([output1_, output2_,output3_,output4_], 1, name="output_")
#     W_out_ = tf.Variable(tf.random_uniform([hidden_dims*text_length*4,classes],-1.0, 1.0), name="W_out")
#     print("W_out: ",W_out_.shape)
#     b_out_ = tf.Variable(tf.zeros([classes,], dtype=tf.float32), name="b_out")
#     logits_ = tf.add(tf.matmul(output_,W_out_), b_out_, name="logits")
#     print("Logits: ",logits_.shape)

with tf.name_scope("Cost_Function"):
    # Full softmax loss for training / scoring
    per_example_loss_ = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_type_, 
                                                                   logits=logits_,
                                                                   name="per_example_loss")
    loss_ = tf.reduce_mean(per_example_loss_, name="loss")
    
with tf.name_scope("Prediction"):
    pred_proba_ = tf.nn.softmax(logits_, name="pred_proba")
    pred_max_ = tf.argmax(logits_, 1, name="pred_max")
#     pred_max_ei = tf.argmax(logits_[:,:2], 1, name="pred_max_ei")
#     pred_max_ = tf.concat([tf.argmax(logits_[:,:2], 1, name="pred_max_ei"),
#                           tf.argmax(logits_[:,2:4], 1, name="pred_max_ns"),
#                           tf.argmax(logits_[:,4:6], 1, name="pred_max_ft"),
#                           tf.argmax(logits_[:,6:8], 1, name="pred_max_jp")], 1, name="max_mbti_bin")

#     pred_random_ = tf.reshape(tf.multinomial(tf.reshape(logits_ , [-1, classes]), 
#                                                           1, 
#                                                           output_dtype=tf.int32, 
#                                                           name="pred_random"),
#                                            [batch_size_,1])
    print("Pred Prob Shape: ",pred_proba_.shape)
    print("Pred Max Shape: ",pred_max_ei.shape)
    print("Sampling Shape: ",pred_random_.shape)

    
with tf.name_scope("Train"):
    learning_rate_ = tf.placeholder(tf.float32, name="learning_rate")
    optimizer_ = tf.train.AdamOptimizer(learning_rate_)
    gradients, variables = zip(*optimizer_.compute_gradients(loss_))
    gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
    train_step_ = optimizer_.apply_gradients(zip(gradients, variables))

# Initializer step
init_ = tf.global_variables_initializer()

Embedded Input:  (?, ?, 300)
LSTM Cell output shape:  (?, ?, 300)
flattened output shape:  (?, 30000)
W_out:  (30000, 8)
Logits:  (?, 8)
Pred Prob Shape:  (?, 8)
Pred Max Shape:  (?,)
Sampling Shape:  (?, 1)


In [14]:
# tensorboard --logdir="tf_graph" --port 6006
summary_writer = tf.summary.FileWriter("tf_graph", 
                                       tf.get_default_graph())

## Train Model

In [77]:
# create batched arrays of size batch_size based on input x and input y
def pad_np_array(example_ids, max_len=35, pad_id=0):
    arr = np.full([len(example_ids), max_len], pad_id, dtype=np.int32)
    ns = np.zeros([len(example_ids)], dtype=np.int32)
    for i, ids in enumerate(example_ids):
        cpy_len = min(len(ids), max_len)
        arr[i,:cpy_len] = ids[:cpy_len]
        ns[i] = cpy_len
    return arr, ns

def batch_generator(x, y, batch_size):
    for i in range(0, len(x), batch_size):
        # padd the batch
        x_batch = x[i:i+batch_size]
        x_padded, _ = pad_np_array(x_batch, max_len=text_length, pad_id=0)
        yield (x_padded, y[i:i+batch_size]) # returns tuple of batched x, batched y

def train_batch(session, xbatch, ybatch, learning_rate):
    feed_dict = {x_text_:xbatch, #np array of texts
                 y_type_:ybatch, #np array of types
                 learning_rate_:learning_rate}
    c, _ = session.run([loss_, train_step_],
                       feed_dict=feed_dict)
    return c

In [79]:
print_interval = 1000

np.random.seed(88)

session = tf.Session()
session.run(init_)

#Override for test training
batch_size = 50
num_epochs = 5

t0 = time.time()
for epoch in range(1,num_epochs+1):
    t0_epoch = time.time()
    epoch_cost = 0.0
    total_batches = 0
    print ("")
    for i, (x,y) in enumerate(batch_generator(x_train[:1000], y_train[:1000], batch_size)):
        if (i % print_interval == 0):
            print("[epoch %d] seen %d batches" % (epoch, i))
            
        epoch_cost += train_batch(session, x, y, learning_rate)
        total_batches = i + 1

    avg_cost = epoch_cost / total_batches
    print("[epoch %d] Completed %d batches in %s" % (epoch, i, utils.pretty_timedelta(since=t0_epoch)))
    print("[epoch %d] Average cost: %.03f" % (epoch, avg_cost,))


[epoch 1] seen 0 batches
[epoch 1] Completed 19 batches in 0:00:39
[epoch 1] Average cost: 454.928

[epoch 2] seen 0 batches
[epoch 2] Completed 19 batches in 0:00:33
[epoch 2] Average cost: 713.758

[epoch 3] seen 0 batches
[epoch 3] Completed 19 batches in 0:00:33
[epoch 3] Average cost: 757.900

[epoch 4] seen 0 batches
[epoch 4] Completed 19 batches in 0:00:36
[epoch 4] Average cost: 764.891

[epoch 5] seen 0 batches
[epoch 5] Completed 19 batches in 0:00:40
[epoch 5] Average cost: 788.423


## Evaluate Model

In [80]:
def predict_type(session,x):
    inputs = {x_text_:x,#np array of texts
             isTrain_: False} 
    pred = session.run([pred_max_],feed_dict=inputs)
    return pred # batch x 1

In [81]:
# Accuracy on train set
train_accuracy=[]
for i, (x,y) in enumerate(batch_generator(x_train[:1000], y_train_id[:1000], batch_size)):
    pred = predict_type(session,x)
    train_accuracy.append((pred[0] == y).mean())
print("Train accuracy: " ,np.mean(train_accuracy))

/Users/linhtran/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


AttributeError: 'bool' object has no attribute 'mean'

In [23]:
# Accuray on test set
test_accuracy = []
for i, (x,y) in enumerate(batch_generator(x_test[:100], y_test_id[:100], batch_size)):
    pred = predict_type(session,x)
    test_accuracy.append((pred[0] == y).mean())
print("Test accuracy: ",np.mean(test_accuracy))

Test accuracy:  0.161


In [39]:
def score_batch(session, x, y):
    feed_dict = {x_text_:x,
                 y_type_:y}
    return session.run(loss_, feed_dict=feed_dict)

def score_dataset(x, y):
    total_cost = 0.0
    total_batches = 0
    for (x,y) in batch_generator(x, y, 5):
        total_cost += score_batch(session, x, y)
        total_batches += 1

    return total_cost / total_batches

In [40]:
print ("Train set perplexity: %.03f" % np.exp(score_dataset(x_train[:5000],y_train[:5000])))
print ("Test set perplexity: %.03f" % np.exp(score_dataset(x_test[:2000],y_test[:2000])))

Train set perplexity: 9.097
Test set perplexity: 41.244
